<a href="https://colab.research.google.com/github/volkfox/DP_similary_score/blob/main/Mturk/Mturk-stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install urllib3!pip install krippendorff

In [17]:
import csv
import random
import pandas as pd
import numpy as np
import requests

url = 'https://github.com/volkfox/DP_similary_score/blob/main/Mturk/mturk-final-python.csv'

#with open(response, newline='') as csvfile:
with requests.Session() as s:
    
    download = s.get(url)
    decoded_content = download.content.decode('utf-8')
    
    turkreader = csv.reader( decoded_content, delimiter=';', quotechar='|')
    next(turkreader, None)

In [18]:
df = pd.read_csv('https://raw.githubusercontent.com/volkfox/DP_similary_score/main/Mturk/mturk-final-python.csv', error_bad_lines=False) 

ratings = df[['Rating 1', 'Rating 2', 'Rating 3', 'Sim ']].to_numpy()
worksheet = np.append(ratings, np.zeros([len(ratings), 2]), 1)

for row in worksheet:
    withold = random.randrange(3)
    average = 0
    for i in range(3):
         if i == withold:
            continue
         average += row[i]
    average /= 2
    diff_human = abs(row[withold]-average)
    diff_sim = abs(row[withold]-row[3])
    row[4] = 0 if diff_human > diff_sim else 1 # where sim underperforms
    row[5] = diff_human if diff_human > diff_sim else diff_sim 
        
        
print("One random selection round results, will change at every run: \n")
print(f"Average percent of Sim giving a better prediction on average of two humans: {np.mean(worksheet[:,4])}")
print(f"Average difference of Sim prediction to human when it was worse: {np.mean(worksheet[worksheet[:,4]==1, 5])}")
print(f"Average difference of Human prediction to sim when it was worse: {np.mean(worksheet[worksheet[:,4]==0, 5])}")


One random selection round results, will change at every run: 

Average percent of Sim giving a better prediction on average of two humans: 0.5277777777777778
Average difference of Sim prediction to human when it was worse: 1.9511364360926315
Average difference of Human prediction to sim when it was worse: 1.8805882352941177


In [19]:
# Statements printed for cases where Sim did worse work than a human

for index, row in enumerate(worksheet):
    if row[4] == 0:
        print(f' topic: {df.iloc[index, 4]}, \n statement: {df.iloc[index,5]} \n' )

 topic:  To a teaching faculty as opposed to research faculty. There's this person just wasn't doing research. So maybe we need not only research faculty and teaching faculty, but maybe we need impact faculty. Is there such a thing as impact faculty and then you can form it., 
 statement: The New School‚Äö√Ñ√∂‚àö√ë‚àö¬•s formal criteria for faculty hiring and promotion should value activities to accelerate impact along with research and teaching and have the Faculty Senate and the Provost appoint a committee to consider these implications. , 

 topic: I think that on paper the sounds like a really good idea, but I think there needs to be a system in place where you know the ways that this money can be spent is kind of controlled for because I mean if you think about it thousand dollars a month can be spent in so many different ways and there's really no way of knowing if this money is going to be put into like a productive use. So I mean we would have to look at you know the system to 

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/volkfox/DP_similary_score/main/Mturk/mturk-final-python.csv', error_bad_lines=False) 

ratings = df[['Rating 1', 'Rating 2', 'Rating 3', 'Sim ']].to_numpy()
worksheet = np.append(ratings, np.zeros([len(ratings), 1]), 1)
worksheet = np.append(worksheet, np.zeros([len(ratings), 1]), 1)

numruns = 1000
run_results = np.zeros([numruns, 3])

for run in range(numruns):
        for row in worksheet:
            withold = random.randrange(3)
            average = 0
            for i in range(3):
                if i == withold:
                    continue
                average += row[i]
            average /= 2
            diff_human = abs(row[withold]-average)
            diff_sim = abs(row[withold]-row[3])
            row[4] = 0 if diff_human > diff_sim else 1 # where sim underperforms
            row[5] = diff_human if diff_human > diff_sim else diff_sim 
        
        run_results[run, 0] = np.mean(worksheet[:,4])
        run_results[run, 1] = np.mean(worksheet[worksheet[:,4]==1, 5])
        run_results[run, 2] = np.mean(worksheet[worksheet[:,4]==0, 5])


In [25]:
print(f"Aggregate statistics over 1,000 runs:")
print(f"Percent of Sim giving a better prediction on average of two humans: {np.mean(run_results[:,0])}, std: {np.std(run_results[:,0])}")
print(f"Difference of Sim prediction to human when it was worse: {np.mean(run_results[:, 1])}, std: {np.std(run_results[:,1])}")
print(f"Difference of Human prediction to sim when it was worse: {np.mean(run_results[:, 2])}, std: {np.std(run_results[:,2])}")


Aggregate statistics over 1,000 runs:
Percent of Sim giving a better prediction on average of two humans: 0.5123027777777778, std: 0.008719709032833834
Difference of Sim prediction to human when it was worse: 1.9395676308620549, std: 0.024294313722934426
Difference of Human prediction to sim when it was worse: 1.8744001608575709, std: 0.022954016618712318


In [9]:
# Krippeddorf's alpha: customary over 0.8 to indicate agreement
# The result of 0.01 indicates human reviewers are often in disagreement

import krippendorff
krippendorff.alpha(ratings[:,0:3].T)

0.01262983700596243